In [1]:
import pandas as pd
import numpy as np

training_raw = pd.read_csv(r"C:\Users\zande\Documents\Ryerson\Capstone\Data\bow_train.csv", encoding = "utf-8", low_memory=False)
test_raw = pd.read_csv(r"C:\Users\zande\Documents\Ryerson\Capstone\Data\bow_test.csv", encoding = "utf-8", low_memory=False)

training_data = training_raw.loc[:,["title", "sentiment"]]
testing_data = test_raw.loc[:,["title", "sentiment"]]


In [2]:
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer
import numpy as np

nltk.download("punkt")

stemmer = PorterStemmer()
def stem_words(words, stemmer):
    stemmed = []
    for w in words:
        stemmed.append(stemmer.stem(w))
    return stemmed

def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    tokens = nltk.word_tokenize(text)
    stems = stem_words(tokens, stemmer)
    return stems

vectorizer = CountVectorizer(analyzer = 'word', tokenizer = tokenize, lowercase = True,
                             stop_words = 'english', max_features = 1000)  # top 1000 features by frequency
# want all possible words in our corpus (labeled/train and unlabeled/test)
corpus_all = vectorizer.fit_transform(list(training_data["title"]) + list(testing_data["title"])).toarray()

corpus_vocab = vectorizer.get_feature_names() # all words

dist = np.sum(corpus_all, axis=0) # freq of tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zande\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test  = train_test_split(corpus_all[:len(training_data)], training_data["sentiment"],
                                                     train_size=0.90, random_state=416)

log_reg_model = LogisticRegression()
log_reg_model = log_reg_model.fit(X=X_train, y=y_train)
y_pred = log_reg_model.predict(X_test)

print(classification_report(y_test, y_pred))


C:\Users\zande\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


             precision    recall  f1-score   support

          0       0.79      0.83      0.81      1198
          1       0.83      0.79      0.81      1241

avg / total       0.81      0.81      0.81      2439



In [4]:
log_reg_model_2 = LogisticRegression()
log_reg_model_2.fit(X = corpus_all[:len(training_data)], y=training_data["sentiment"])

test_pred = log_reg_model_2.predict(corpus_all[len(training_data):])
test_labels = list(testing_data["sentiment"])
print(classification_report(test_pred, test_labels))

             precision    recall  f1-score   support

          0       0.87      0.87      0.87       430
          1       0.87      0.87      0.87       430

avg / total       0.87      0.87      0.87       860

